In [2]:
from common.inputreader import InputReader, PuzzleWrapper

puzzle = PuzzleWrapper(year=int("2022"), day=int("19"))

puzzle.header()

# Not Enough Minerals

[Open Website](https://adventofcode.com/2022/day/19)

In [36]:

class Cost:
    def __init__(self, robot: str, costs: dict):
        self.robot = robot
        self.costs = costs

    def can_produce(self, state: dict) -> bool:
        for type in self.costs:
            if state[type] < self.costs[type]:
                return False
        return True

    def __repr__(self):
        return (f"Cost(robot={self.robot}, "
                f"costs={self.costs})")


# helper functions
class Blueprint:
    def __init__(self, costs: dict):
        self.costs = costs

    def __repr__(self):
        return (f"Blueprint(costs={self.costs})")


def domain_from_input(input: InputReader) -> set:
    lines = input.lines_as_str()
    # remove empty lines
    lines = [line for line in lines if line]
    blueprints = set()

    for i in range(0, len(lines), 5):
        costs = {}
        # parse ore robot using pattern
        line = lines[i + 1].strip(".").split()
        costs[line[1]] = {
            line[-1]: int(line[-2]),
        }

        # parse clay robot using pattern
        line = lines[i + 2].strip(".").split()
        costs[line[1]] = {
            line[-1]: int(line[-2]),
        }

        # parse obsidian robot using pattern
        line = lines[i + 3].strip(".").split()
        costs[line[1]] = {
            line[-1]: int(line[-2]),
            line[-4]: int(line[-5]),
        }

        # parse geode robot using pattern
        line = lines[i + 4].strip(".").split()
        costs[line[1]] = {
            line[-1]: int(line[-2]),
            line[-4]: int(line[-5]),
        }

        for type in costs.keys():
            costs[type] = Cost(type, costs[type])

        blueprint = Blueprint(costs)
        blueprints.add(blueprint)

    return blueprints


test_input = domain_from_input(puzzle.get_code_block(0))
print(test_input)

{Blueprint(costs={'ore': Cost(robot=ore, costs={'ore': 2}), 'clay': Cost(robot=clay, costs={'ore': 3}), 'obsidian': Cost(robot=obsidian, costs={'clay': 8, 'ore': 3}), 'geode': Cost(robot=geode, costs={'obsidian': 12, 'ore': 3})}), Blueprint(costs={'ore': Cost(robot=ore, costs={'ore': 4}), 'clay': Cost(robot=clay, costs={'ore': 2}), 'obsidian': Cost(robot=obsidian, costs={'clay': 14, 'ore': 3}), 'geode': Cost(robot=geode, costs={'obsidian': 7, 'ore': 2})})}


In [43]:
# test case (part 1)
def part_1(reader: InputReader, debug: bool) -> int:
    blueprints = domain_from_input(reader)
    total_geodes = 0
    for blueprint in blueprints:
        if debug:
            print(blueprint)

        build_order = [
            "geode",
            "obsidian",
            "clay",
            "ore"
        ]

        state = {
            "robots": {
                "ore": 1,
                "clay": 0,
                "obsidian": 0,
                "geode": 0,
            },
            "supplies": {
                "ore": 0,
                "clay": 0,
                "obsidian": 0,
                "geode": 0
            }
        }
        for i in range(24):
            # produce ore
            for type in state["robots"]:
                state["supplies"][type] += state["robots"][type]

            # product robots if possible
            for type in build_order:
                if blueprint.costs[type].can_produce(state["supplies"]):
                    for cost_type in blueprint.costs[type].costs:
                        state["supplies"][cost_type] -= blueprint.costs[type].costs[cost_type]
                    state["robots"][type] += 1

            if debug:
                print(f"state after round {i + 1}")
                print(state)

        total_geodes += state["supplies"]["geode"]
        break

    return total_geodes


result = part_1(puzzle.example(0), True)
print(result)
assert result == 33

Blueprint(costs={'ore': Cost(robot=ore, costs={'ore': 4}), 'clay': Cost(robot=clay, costs={'ore': 2}), 'obsidian': Cost(robot=obsidian, costs={'clay': 14, 'ore': 3}), 'geode': Cost(robot=geode, costs={'obsidian': 7, 'ore': 2})})
state after round 1
{'robots': {'ore': 1, 'clay': 0, 'obsidian': 0, 'geode': 0}, 'supplies': {'ore': 1, 'clay': 0, 'obsidian': 0, 'geode': 0}}
state after round 2
{'robots': {'ore': 1, 'clay': 1, 'obsidian': 0, 'geode': 0}, 'supplies': {'ore': 0, 'clay': 0, 'obsidian': 0, 'geode': 0}}
state after round 3
{'robots': {'ore': 1, 'clay': 1, 'obsidian': 0, 'geode': 0}, 'supplies': {'ore': 1, 'clay': 1, 'obsidian': 0, 'geode': 0}}
state after round 4
{'robots': {'ore': 1, 'clay': 2, 'obsidian': 0, 'geode': 0}, 'supplies': {'ore': 0, 'clay': 2, 'obsidian': 0, 'geode': 0}}
state after round 5
{'robots': {'ore': 1, 'clay': 2, 'obsidian': 0, 'geode': 0}, 'supplies': {'ore': 1, 'clay': 4, 'obsidian': 0, 'geode': 0}}
state after round 6
{'robots': {'ore': 1, 'clay': 3, 'ob

AssertionError: 

In [ ]:
# real case (part 1)
result = part_1(puzzle.input(), False)
print(result)

In [ ]:
# test case (part 2)
def part_2(reader: InputReader, debug: bool) -> int:
    lines = domain_from_input(reader)
    if debug:
        print(lines)
    return 0


result = part_2(puzzle.example(0), True)
print(result)
assert result == 0

In [ ]:
# real case (part 2)
result = part_2(puzzle.input(), False)
print(result)

In [ ]:
# print easters eggs
puzzle.print_easter_eggs()